# **Auto encoders with Bidirectional LSTM**

In [ ]:
pip install tensorflow --upgrade

In [ ]:
!pip install tensorflow-addons==0.16.1
!pip install --upgrade tensorflow tensorflow_addons keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.1.5
    Uninstalling typeguard-4.1.5:
      Successfully uninstalled typeguard-4.1.5
  Attempting uninstall: tensorflow_addons
    Found existing installation: tensorflow-addons 0.16.1
    Uninstalling tensorflow-addons-0.16.1:
      Successfully uninstalled tensorf

In [ ]:
!pip install tensorflow-addons==0.16.1
!pip install --upgrade tensorflow tensorflow_addons keras

  Using cached tensorflow_addons-0.16.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: tensorflow-addons
    Found existing installation: tensorflow-addons 0.22.0
    Uninstalling tensorflow-addons-0.22.0:
      Successfully uninstalled tensorflow-addons-0.22.0
  Using cached tensorflow_addons-0.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (612 kB)
  Attempting uninstall: tensorflow_addons
    Found existing installation: tensorflow-addons 0.16.1
    Uninstalling tensorflow-addons-0.16.1:
      Successfully uninstalled tensorflow-addons-0.16.1


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DataSet_RTP.xlsx to DataSet_RTP.xlsx


In [ ]:
import pandas as pd
import io
import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa

from keras import Sequential
from keras import layers
from keras.models import Model
from keras.layers import LSTM, Bidirectional,RepeatVector, BatchNormalization, Dropout, Dense, Flatten, Conv1D
from keras.layers import MaxPooling1D, GRU, Input,Masking, Concatenate, dot
from keras.optimizers import Adam, SGD
from keras.losses import MeanAbsoluteError
from keras.metrics import RootMeanSquaredError
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l1, l2
from sklearn.model_selection import train_test_split
#from tensorflow.keras.optimizers import legacy
from sklearn import metrics
import numpy as np

In [ ]:
df = pd.read_excel(io.BytesIO(uploaded['DataSet_RTP.xlsx']))
df.head()

,Feat 1,Feat 2,Feat 3,Feat 4,Feat 5,Feat 6,Feat 7,Feat 8,Feat 9,Feat 10,...,Feat 28,Feat 29,Feat 30,Feat 31,Feat 32,Feat 33,Feat 34,Feat 35,Feat 36,Target
0,34.99,34.99,318.41,83.83,61.06,72.48,77.58,81.85,71.15,71.17,...,-116.92,-89.01,-100.14,-105.19,-109.42,-80.33,-80.36,-74.08,-77.19,63.18
1,63.18,19.69,18.70,33.80,65.58,82.33,75.85,91.05,286.20,197.35,...,-37.51,-74.02,-111.84,-85.63,-103.06,-315.68,-224.84,-80.22,-85.10,66.96
2,66.96,35.19,33.78,26.20,26.22,26.18,21.43,21.43,17.53,17.54,...,-28.73,-28.76,-28.67,-23.23,-23.23,-18.77,-18.78,-18.78,-28.61,21.42
3,21.42,21.43,14.98,1.20,29.57,26.15,1.19,6.52,17.53,17.53,...,-0.06,-42.01,-28.61,-0.06,-6.17,-18.76,-18.76,0.00,0.00,21.47
4,21.47,10.86,1.14,1.14,4.53,1.13,1.12,1.13,1.13,1.12,...,0.00,-3.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
y = df.Target - min(df.Target) +1
y= np.log1p(y)
x = df.drop("Target", axis =1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(17356, 36)
(17356,)
(4340, 36)
(4340,)


In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.125, random_state=42)

In [ ]:
input_shape = (36, 1)
encoding_dim = 9

In [ ]:
# Encoder
inputs = Input(shape=input_shape)
encoded = Bidirectional(LSTM(encoding_dim, activation='relu'))(inputs)


In [ ]:
# Decoder
decoded = RepeatVector(input_shape[0])(encoded)
decoded = Bidirectional(LSTM(1, return_sequences=True, activation='relu'))(decoded)


In [ ]:

autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

In [ ]:

# Train the autoencoder first
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(x_train, x_train, epochs=100, batch_size=64, validation_data=(x_val, x_val), callbacks = [callback])


Epoch 1/100
297/297 [==============================] - 24s 57ms/step - loss: 1.0612 - val_loss: 0.7187
Epoch 2/100
297/297 [==============================] - 16s 54ms/step - loss: 1.0158 - val_loss: 0.6793
Epoch 3/100
297/297 [==============================] - 16s 54ms/step - loss: 0.9590 - val_loss: 0.6464
Epoch 4/100
297/297 [==============================] - 19s 63ms/step - loss: 0.9409 - val_loss: 0.5957
Epoch 5/100
297/297 [==============================] - 29s 96ms/step - loss: 0.9302 - val_loss: 0.5904
Epoch 6/100
297/297 [==============================] - 32s 107ms/step - loss: 0.9253 - val_loss: 0.5777
Epoch 7/100
297/297 [==============================] - 20s 68ms/step - loss: 0.9189 - val_loss: 0.5731
Epoch 8/100
297/297 [==============================] - 16s 54ms/step - loss: 0.9141 - val_loss: 0.5666
Epoch 9/100
297/297 [==============================] - 16s 54ms/step - loss: 0.9065 - val_loss: 0.5634
Epoch 10/100
297/297 [==============================] - 16s 55ms/step - 

In [ ]:

x_train = x_train.reshape((-1, x_train.shape[1], 1))
x_val = x_val.reshape((-1, x_val.shape[1], 1))


In [ ]:

# Then use the encoder to transform data
x_train_encoded = encoder.predict(x_train)
x_val_encoded = encoder.predict(x_val)

85/85 [==============================] - 1s 8ms/step


In [ ]:
print(x_train[6:10])

[[[-0.29323505]
  [-0.34008254]
  [-0.35621547]
  [-0.36902192]
  [-0.35334528]
  [-0.36760703]
  [-0.36723381]
  [-0.26381116]
  [-0.2802227 ]
  [-0.30696921]
  [-0.33504985]
  [-0.25734539]
  [-0.09596522]
  [-0.09094318]
  [-0.07252147]
  [-0.07147817]
  [-0.08287386]
  [-0.08257754]
  [-0.09052856]
  [-0.03760022]
  [-0.03430738]
  [-0.05802349]
  [-0.07926217]
  [-0.055003  ]
  [ 0.126676  ]
  [ 0.13520661]
  [ 0.12062345]
  [ 0.11566832]
  [ 0.12560367]
  [ 0.11710528]
  [ 0.13264031]
  [ 0.13120946]
  [ 0.12109672]
  [ 0.11611075]
  [ 0.12320492]
  [ 0.13405123]]

 [[-0.12222205]
  [-0.13197918]
  [-0.14622574]
  [-0.18372255]
  [-0.2066187 ]
  [-0.22310185]
  [-0.21912177]
  [-0.18007081]
  [-0.20543535]
  [-0.24387731]
  [-0.26358211]
  [-0.30626825]
  [ 0.0693103 ]
  [ 0.05550886]
  [ 0.05112753]
  [ 0.04201137]
  [ 0.01911045]
  [ 0.00473355]
  [ 0.02353023]
  [ 0.03320012]
  [ 0.02180048]
  [-0.00976097]
  [-0.02018469]
  [-0.07471993]
  [ 0.126676  ]
  [ 0.13520661]
  [ 0.

In [ ]:
x_train_encoded_flattened = x_train_encoded.flatten()
#x_train_diff = pd.DataFrame({'Actual value': x_train.flatten(), 'Predicted value': x_train_encoded_flattened})
print(x_train_encoded[6:10])

[[2.38441885e-06 5.27435839e-01 8.51038098e-01 2.64747947e-01
  2.10299164e-01 0.00000000e+00 2.21767068e+00 2.91118312e+00
  0.00000000e+00 1.06853150e-01 1.37655521e-02 1.91565543e-01
  1.20584559e+00 3.18119764e-01 5.15454868e-03 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.51710423e-06 5.02315581e-01 7.95340300e-01 2.05030471e-01
  2.25157425e-01 0.00000000e+00 1.84643722e+00 2.51941085e+00
  0.00000000e+00 7.68604651e-02 8.13970529e-03 1.28905222e-01
  1.04428089e+00 3.01147580e-01 1.29374517e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.56932958e-06 5.09555101e-01 8.09877694e-01 2.19073549e-01
  2.21655145e-01 0.00000000e+00 1.93643212e+00 2.61604166e+00
  0.00000000e+00 8.68929923e-02 9.41367541e-03 1.57384470e-01
  1.09101653e+00 3.04908037e-01 1.04640517e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00476835e-04 6.72766626e-01 5.79444706e-01 2.28637174e-01
  4.52073030e-02 1.01894339e-05 1.80428636e+00 2.43812108e+00
  4.70872976e-07 2.72589666e-03 1

In [ ]:
# Encoded input
encoded_input = Input(shape=(18,))
x = RepeatVector(36)(encoded_input)
x = Bidirectional(LSTM(units=100, return_sequences=True, activation="relu", recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))(x)
x = Flatten()(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(0.1)(x)
predictions = Dense(1)(x)

combined_model = Model(encoded_input, predictions)


In [ ]:
epoch = 100
batch_size = 64
# Define cyclic learning rate, optimizer, and callback
steps_per_epoch = len(x_train) // batch_size
cyclic_lr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=1e-04,
                                                maximal_learning_rate=1e-02,
                                                scale_fn=lambda x: 1/(2**(x-1)),
                                                step_size=6 * steps_per_epoch)
optimizer = Adam(learning_rate=cyclic_lr, amsgrad=True)


# Compile and train the combined model
combined_model.compile(optimizer=optimizer, loss='mean_absolute_error')
combined_model.fit(x_train_encoded, y_train, validation_data=(x_val_encoded, y_val), epochs=epoch, batch_size=batch_size, callbacks=[callback])

Epoch 1/100
297/297 [==============================] - 65s 195ms/step - loss: 1.8038 - val_loss: 0.4296
Epoch 2/100
297/297 [==============================] - 61s 206ms/step - loss: 0.5490 - val_loss: 0.4864
Epoch 3/100
297/297 [==============================] - 59s 199ms/step - loss: 0.4366 - val_loss: 0.5083
Epoch 4/100
297/297 [==============================] - 58s 195ms/step - loss: 0.4217 - val_loss: 0.3055
Epoch 5/100
297/297 [==============================] - 55s 186ms/step - loss: 0.3758 - val_loss: 0.0784
Epoch 6/100
297/297 [==============================] - 57s 191ms/step - loss: 0.3678 - val_loss: 0.1916
Epoch 7/100
297/297 [==============================] - 55s 185ms/step - loss: 0.3142 - val_loss: 0.1844
Epoch 8/100
297/297 [==============================] - 56s 188ms/step - loss: 0.2814 - val_loss: 0.1534
Epoch 9/100
297/297 [==============================] - 56s 187ms/step - loss: 0.2591 - val_loss: 0.0575
Epoch 10/100
297/297 [==============================] - 55s 187m

In [ ]:
x_test = x_test.reshape((-1, x_test.shape[1], 1))
# Use encoder to transform x_test and make predictions
x_test_encoded = encoder.predict(x_test)
y_predict = combined_model.predict(x_test_encoded)

y_predict = y_predict.flatten()
print(y_predict)
diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_predict})
print(diff)

136/136 [==============================] - 4s 23ms/step
[6.7338223 6.72738   6.7351103 ... 6.764512  6.7435694 6.7345276]
       Actual value  Predicted value
2408       6.733878         6.733822
3045       6.721570         6.727380
812        6.724145         6.735110
9814       6.720401         6.728891
6231       6.727348         6.733798
...             ...              ...
12376      6.746154         6.753952
14255      6.734865         6.735458
601        6.759255         6.764512
17889      6.743541         6.743569
2167       6.734782         6.734528

[4340 rows x 2 columns]


In [ ]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [ ]:
meanAbErr = metrics.mean_absolute_error(y_test, y_predict)
meanSqErr = metrics.mean_squared_error(y_test, y_predict)
rootMeanSqErr = np.sqrt(metrics.mean_squared_error(y_test, y_predict))
mape= MAPE(y_test,y_predict)
#print('R squared: {:.2f}'.format(regr.score(x,y)*100))
print('Mean Absolute Error:', meanAbErr)
print('Mean Absolute percentage Error:', mape)
#print('Mean Square Error:', meanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)

Mean Absolute Error: 0.011958601959053506
Mean Absolute percentage Error: 0.17597641854521134
Root Mean Square Error: 0.03590643020393213
